In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
import os
import pandas as pd

In [6]:
# get the raw scraped webpages

# Define the list of allowed URLs (whitelist)
allowed_urls = [
    "https://www.polivy-hcp.com/newly-diagnosed/rchp/efficacy/trial-results.html",
    "https://www.polivy-hcp.com/newly-diagnosed/rchp/safety/important-safety-information.html",
    "https://www.polivy-hcp.com/newly-diagnosed/rchp/safety/summary.html",
    "https://www.polivy-hcp.com/newly-diagnosed/rchp/efficacy/polarix-trial.html#trial-design",
    "https://www.epkinlyhcp.com/dlbcl/study-design",
    "https://www.epkinlyhcp.com/dlbcl/clinical-trial-results",
    "https://www.epkinlyhcp.com/important-safety-information",
    "https://www.epkinlyhcp.com/dlbcl/adverse-reactions"
]

# Grab the HTML using Selenium and extract text with beautifulsoup
def parse_selenium(url):
    try:
        driver = webdriver.Chrome()  # preferred browser
        driver.get(url)

        # Get rendered HTML content
        html_content = driver.page_source
        driver.quit()

        soup = BeautifulSoup(html_content, 'html.parser')

        # Extract text from the page
        text = soup.get_text()

        # Create a filename based on the URL
        filename = url.split('/')[2].rstrip("-hcp.com").lstrip("www.") + "_" + url.split('/')[-1].rstrip(".html") + '.txt'

        # Write raw text to a file
        if not os.path.exists("text/"):
            os.mkdir("text/")
            
        with open("text/"+filename, 'w', encoding='utf-8') as f:
            f.write(text)

    except Exception as e:
        print(f"Error fetching url: {url}, {e}")

# Iterate through the URLs and parse each one
for url in allowed_urls:
    parse_selenium(url)


In [10]:
# fast cleanup of raw text 

# helper function to remove all the newlines that mess up embeddings
def remove_newlines(txt):
    txt = txt.str.replace('\n', ' ')
    txt = txt.str.replace('\\n', ' ')
    txt = txt.str.replace('  ', ' ')
    txt = txt.str.replace('  ', ' ')
    return txt

# Create a list to store the text files
texts=[]

# Get all the text files in the text directory
for file in os.listdir("text/"):
    print(file)

    # Open the file and read the text
    with open("text/"+ file, "r") as f:
        text = f.read()
        texts.append((file, text))

# Create a dataframe from the list of texts
df = pd.DataFrame(texts, columns = ['fname', 'text'])

# Remove the newlines
df['text'] = df.fname + ". " + remove_newlines(df.text)

# save scraped dataframe as a tsv

if not os.path.exists("processed"):
    os.mkdir("processed")
        
df.to_csv('processed/scraped.tsv', index=False, sep='\t')
df.head()

polivy_summary.txt
epkinly_study-design.txt
polivy_polarix-trial.html#trial-design.txt
polivy_trial-results.txt
epkinly_important-safety-information.txt
epkinly_clinical-trial-results.txt
epkinly_adverse-reactions.txt
polivy_important-safety-information.txt


,fname,text
0,polivy_summary.txt,polivy_summary.txt. POLIVY® (polatuzumab vedo...
1,epkinly_study-design.txt,epkinly_study-design.txt. DLBC...
2,polivy_polarix-trial.html#trial-design.txt,polivy_polarix-trial.html#trial-design.txt. P...
3,polivy_trial-results.txt,polivy_trial-results.txt. POLARIX Trial Resul...
4,epkinly_important-safety-information.txt,epkinly_important-safety-information.txt. ...
